<a href="https://colab.research.google.com/github/yenlung/Generative-AI-Book/blob/main/%E5%86%92%E9%9A%AA03_%E3%80%8A%E7%B4%85%E6%A8%93%E5%A4%A2%E3%80%8B%E7%94%9F%E6%88%90%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U tensorflow-probability
!pip install typing-extensions --upgrade
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 19.4 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-probability
    Found existing installation: tensorflow-probability 0.22.0
    Uninstalling tensorflow-probability-0.22.0:
      Successfully uninstalled tensorflow-probability-0.22.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. 讀入套件

In [3]:
import tensorflow as tf
import pickle

from tensorflow.keras.models import load_model, model_from_json

### 2. 讀入訓練好的 RNN 紅樓夢生成器

In [4]:
!wget --no-check-certificate \
    https://github.com/yenlung/Python-AI-Book/blob/main/dream_rnn.zip?raw=true \
    -O /content/dream_rnn.zip

--2023-12-13 12:11:17--  https://github.com/yenlung/Python-AI-Book/blob/main/dream_rnn.zip?raw=true
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/yenlung/Python-AI-Book/raw/main/dream_rnn.zip [following]
--2023-12-13 12:11:17--  https://github.com/yenlung/Python-AI-Book/raw/main/dream_rnn.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/yenlung/Python-AI-Book/main/dream_rnn.zip [following]
--2023-12-13 12:11:17--  https://raw.githubusercontent.com/yenlung/Python-AI-Book/main/dream_rnn.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting 

In [5]:
import os
import zipfile

local_zip = '/content/dream_rnn.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
f = open('architecture.json', 'r')
loaded_model = f.read()
f.close()

In [7]:
model = model_from_json(loaded_model)

In [8]:
model.load_weights("weights.h5")

【更新】TensorFlow 更新後, 我們發現需要這個步驟。

In [9]:
!pip install Keras-Preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.7 MB/s eta 0:00:00


In [10]:
f = open('tokenizer.pkl', 'rb')
tokenizer = pickle.load(f)
f.close()

#### 【已下載紅樓夢生成器請執行這段】

如果你已經在[我的 GitHub](https://github.com/yenlung/Deep-Learning-Basics) 中下載紅樓夢生成模型, 這包括:

1. `dream_rnn` 資料夾, 這是模型和訓練好的權重。
2. `dream_tokenizer2.pkl` 檔案, 這是模型使用的 tokenizer。

存到你的 Google Drive, 放 Colab 程式的地方 (預設是 `Colab Notebooks` 資料夾)。那執行下面這一段, 不用再讀入一次模型資料。

### 3. 打造紅樓夢生成器

In [11]:
max_id = len(tokenizer.word_index)

In [12]:
max_id

4522

接下來是一段文字, 我們用事先訓練好的 tokenizer 換成一段數字, 最後用 one-hot encoding 回傳。

In [13]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences([texts]))-1
    return tf.one_hot(X, max_id)

這段程式主要依輸入的一段文字, 用我們的 model 去預測下一個字。注意像平常的分類問題, 這裡輸出是每人個字出現機率最高的。但都照這樣, 我們輸入同一段文字, 之後出現的文字永遠是一樣的! 常用的手法是去設定 `temperature`, `temperature` 接近 0, 大致上就取機率最高的字; `temperature` 越大就越隨機。太隨機就變成亂數取字! 一般 `temperature`設 1 左右效果最佳。

In [14]:
def next_char(texts, temperature=1):
    X_new = preprocess(texts)
    y_predict = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_predict) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

最後就一段文字進來，再產生 `n_chars` 這麼多個字。我們原本一段文字只能生一個字, 那就一次生一個字, 最後要生多少個字就生多少個字。

原本訓練我們一段是 100 個字去訓練的, 這裡超過 100 字時我們就取最後 100 個字丟入模型。

In [15]:
def complete_text(texts, n_chars=50, temperature=1):
    n_chars=int(n_chars)
    for _ in range(n_chars):
        texts = texts + next_char(texts[-100:], temperature)
    return texts

### 4. 測試紅樓夢生成器

In [16]:
complete_text("自孫悟空從石頭中蹦出來之後，", n_chars=300, temperature=1.2)

1/1 [==============================] - 0s 41ms/step


'自孫悟空從石頭中蹦出來之後，說道他一番方法。原來如今如此，好容易進來了！況且又是水仙一長的？」正說著，笑嘻地說道：「阿彌陀佛！好容易姥姥來了！我恨不得在後門，若是來不肯叫你、媳婦了屋裡的麼？」寶玉點點頭兒了半天，也不聽他起來往，見是那裡的人把寶玉說，不不難語，卻把這個志氣的收了心，令空了原明再單甄寶釵這一番閒話。寶玉看他自己也都不知是寶玉。但我也多八九歲，並沒有執手。三個遇見此事交地，居然，便要細看，仍舊擲下道：「來是你叔叔作文，也是你麼傷心。」那人道：「頭一把兒這個心裡都沒有了我，你見了一面，這還多，他須待管保是聽的了。他別他不錯的。那看你自己跟了一個和尚，我也猜著了八九』字了。但只知沒有這個事，又念了幾首怎麼處才好呢'

### 5. 用 `gradio` 做成一個網路 app

我們準備用 [`gradio`](https://gradio.app/) 套件, 神速做完一個 web app。最酷的是, 最後出現 `https://xxxx.gradio.app` 那個網址, 在你的 Colab 還在執行的時候, 任何人都可以用任何瀏覽器連進來使用!

In [17]:
#!pip install gradio

In [18]:
import gradio as gr

In [19]:
iface = gr.Interface(
    fn=complete_text,
    inputs=[
        "text",
        gr.Slider(minimum=50, maximum=200, value=50, step=1),
        gr.Slider(minimum=0.2, maximum=2, value=1, step=0.2)],
    outputs="text",
    title="紅樓夢生成器",
    description="起個頭, 幫你完成一段紅樓夢。可以改變 temperature, 越小生出的字越固定, 越大越隨機。")
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b96fdc1c138763220f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
